<a href="https://colab.research.google.com/github/rselent/project-canopy-ds/blob/master/notebooks/canopyCreateDatabase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# .ENV SUPPORT

#### (\*\*\*RUN ME FIRST IF USING COLAB***)

In [ ]:
!pip install colab-env -qU

In [ ]:
import colab_env

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
## USE COLAB_ENV TO CREATE/LOAD LOCAL FILE 'VARS.ENV'
## (token was saved to file using this library/method)

## CREATES vars.env in root directory of google Drive

 from colab_env import envvar_handler
# help( envvar_handler.add_env)
# help( envvar_handler.envload)

# BUILDING DATABASE:

#### Setup and Imports:

In [ ]:
%%capture
github_link = 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/source_data/iucn/'
github_link_cites = 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/source_data/cites/'

!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/assessments.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/common_names.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/taxonomy.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/conservation_needed.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/countries.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/habitats.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/research_needed.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/threats.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/usetrade.csv'
!wget -P datasets/ 'https://raw.githubusercontent.com/Lambda-School-Labs/project-canopy-ds/master/data/clean_data/cites_listings.csv'


In [ ]:
%%capture
!pip install --upgrade flask_sqlalchemy

In [ ]:

import os
import flask_sqlalchemy
import sqlalchemy
import psycopg2
import pandas as pd
import numpy as np

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
from sqlalchemy import create_engine


In [ ]:

# ALL VARIABLE CREDENTIALS SET TO PULL FROM ENV FILE
AWS_CANOPY_USER = os.getenv( "AWS_CANOPY_USER")
AWS_CANOPY_PW = os.getenv( "AWS_CANOPY_PW")
AWS_CANOPY_HOST = os.getenv( "AWS_CANOPY_HOST")
AWS_CANOPY_PORT = os.getenv( "AWS_CANOPY_PORT")
AWS_CANOPY_DBNAME = os.getenv( "AWS_CANOPY_DBNAME")

# instantiating above credentials in SQLA postgres URI format
URL = 'postgresql://{}:{}@{}:{}/{}'.format( AWS_CANOPY_USER, 
                                            AWS_CANOPY_PW, 
                                            AWS_CANOPY_HOST, 
                                            AWS_CANOPY_PORT, 
                                            AWS_CANOPY_DBNAME
)

app = Flask( __name__)
app.config[ 'SQLALCHEMY_DATABASE_URI'] = URL
app.config[ 'SQLALCHEMY_TRACK_MODIFICATIONS'] = False

DB = SQLAlchemy( app)
engine = DB.create_engine( URL, {})
curse = engine.connect()

#### Database Schema:

In [ ]:
# putting each table declaration into its own cell, to aid potential troubleshooting

class Assessments( DB.Model):
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), nullable= False, primary_key= True)
    redlistCategory = DB.Column( DB.String(), nullable= False)
    redlistCriteria = DB.Column( DB.String())
    yearPublished = DB.Column( DB.Numeric( 4, 0), nullable= False)
    assessmentDate = DB.Column( DB.String(), nullable= False)
    criteriaVersion = DB.Column( DB.Numeric( 4, 0), nullable= False)
    language = DB.Column( DB.String())
    rationale = DB.Column( DB.String())
    habitat = DB.Column( DB.String())
    threats = DB.Column( DB.String())
    population = DB.Column( DB.String())
    populationTrend = DB.Column( DB.String())
    range = DB.Column( DB.String())
    useTrade = DB.Column( DB.String())
    systems = DB.Column( DB.String())
    conservationActions = DB.Column( DB.String())
    realm = DB.Column( DB.String())
    yearLastSeen = DB.Column( DB.String())
    possiblyExtinct = DB.Column( DB.Boolean())
    possiblyExtinctInTheWild = DB.Column( DB.Boolean())
    scopes = DB.Column( DB.String())

    def __repr__( self):
        return '<Assessment for {}>'.format( self.scientificName)

In [ ]:

class CommonNames( DB.Model):
#    __tablename__ = 'commonNames'          # not sure if this is wise just yet
                                            # (maybe in a later update/release)
    index = DB.Column( DB.Numeric, primary_key= True)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'),          # foreign key assignment rolled back in all tables
                                nullable= False                                         # due to irregular data (species that exist in one table
    )                                                                                   # don't necessarily exist in another, it seems -- violates
    name = DB.Column( DB.String(), nullable= False)                                     # database laws or some such?)
    language = DB.Column( DB.String(), nullable= False)
    main = DB.Column( DB.Boolean, nullable= False)

    def __repr__( self):
        return '<Common names for {}>'.format( self.scientificName)

In [ ]:

class Taxonomy( DB.Model):
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False, primary_key= True
    )
    kingdomName = DB.Column( DB.String(), nullable= False)
    phylumName = DB.Column( DB.String(), nullable= False)
    orderName = DB.Column( DB.String(), nullable= False)
    className = DB.Column( DB.String(), nullable= False)
    familyName = DB.Column( DB.String(), nullable= False)
    genusName = DB.Column( DB.String(), nullable= False)
    speciesName = DB.Column( DB.String(), nullable= False)
    infraType = DB.Column( DB.String())
    infraName = DB.Column( DB.String())
    infraAuthority = DB.Column( DB.String())
    subpopulationName = DB.Column( DB.String())
    authority = DB.Column( DB.String())
    taxonomicNotes = DB.Column( DB.String())

    def __repr__( self):
        return '<Taxonomy for {}>'.format( self.scientificName)

In [ ]:

class ConservationNeeded( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    note = DB.Column( DB.String())

    def __repr__( self):
        return '<Conservation needed for {}>'.format( self.scientificName)

In [ ]:

class Countries( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    presence = DB.Column( DB.String())
    origin = DB.Column( DB.String())
    seasonality = DB.Column( DB.String())
    formerlyBred = DB.Column( DB.String())

    def __repr__( self):
        return '<Countries that {} can be found in>'.format( self.scientificName)

In [ ]:

class Habitats( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    majorImportance = DB.Column( DB.String())
    season = DB.Column( DB.String())
    suitability = DB.Column( DB.String())

    def __repr__( self):
        return '<Habitats that {} can be found in>'.format( self.scientificName)

In [ ]:
## NOT USING PLANTS (yet? keeping just in case)

'''
class Plants( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String())
    name = DB.Column( DB.String())

    def __repr__( self):
        return '<Plant {}'.format( self.scientificName)
'''


In [ ]:

class ResearchNeeded( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    note = DB.Column( DB.String())

    def __repr__( self):
        return '<Research still needed for {}>'.format( self.scientificName)

In [ ]:

class Threats( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    stressCode = DB.Column( DB.String())
    stressName = DB.Column( DB.String())
    ancestry = DB.Column( DB.String())
    ias = DB.Column( DB.String())
    internationalTrade = DB.Column( DB.String())
    scope = DB.Column( DB.String())
    severity = DB.Column( DB.String())
    text = DB.Column( DB.String())
    timing = DB.Column( DB.String())
    virus = DB.Column( DB.String())

    def __repr__( self):
        return '<Threats to {}>'.format( self.scientificName)

In [ ]:

class UseTrade( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    assessmentId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    internalTaxonId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    code = DB.Column( DB.String(), nullable= False)
    name = DB.Column( DB.String(), nullable= False)
    international = DB.Column( DB.String())
    national = DB.Column( DB.String())
    other = DB.Column( DB.String())
    subsistence = DB.Column( DB.String())

    def __repr__( self):
        return '<Trade uses for {}>'.format( self.scientificName)

In [ ]:

class CitesIndex( DB.Model):
    index = DB.Column( DB.Numeric, primary_key= True)
    speciesId = DB.Column( DB.Numeric( 12, 0), nullable= False)
    kingdomName = DB.Column( DB.String())
    phylumName = DB.Column( DB.String())
    orderName = DB.Column( DB.String())
    className = DB.Column( DB.String())
    familyName = DB.Column( DB.String())
    genusName = DB.Column( DB.String())
    speciesName = DB.Column( DB.String())
    subspeciesName = DB.Column( DB.String())
    scientificName = DB.Column( DB.String(), 
                                #DB.ForeignKey( 'assessments.scientificName'), 
                                nullable= False
    )
    author = DB.Column( DB.String())
    rank = DB.Column( DB.String())
    listing = DB.Column( DB.String(), nullable= False)
    party = DB.Column( DB.String())
    listedUnder = DB.Column( DB.String())
    fullNote = DB.Column( DB.String())
    full_note = DB.Column( DB.String())
    countries_distributionFullNames = DB.Column( DB.String())
    countries_distributionISOCodes = DB.Column( DB.String())
    nativeDistributionFullNames = DB.Column( DB.String())
    introducedDistribution = DB.Column( DB.String())
    introducedDistribution_questionmark = DB.Column( DB.String())
    reintroduced_Distribution = DB.Column( DB.String())
    extinctDistribution = DB.Column( DB.String())
    extinctDistribution_questionmark = DB.Column( DB.String())
    distribution_Uncertain = DB.Column( DB.String())

    def __repr__( self):
        return '<CITES listing for {}>'.format( self.scientificName)

#### Working and Building:

(NOTE: section NOT designed to be run sequentially)

In [ ]:
# display tables currently in metadata (being worked on / changes to be committed)

for t in DB.metadata.sorted_tables:
    print( t.name)


assessments
cites_index
common_names
conservation_needed
countries
habitats
research_needed
taxonomy
threats
use_trade


In [ ]:
# clear metadata cache / queue

DB.metadata.clear()

In [ ]:
# drop all tables currently listed in metadata

DB.drop_all()

In [ ]:
# create all tables currently listed in metadata

DB.create_all()

In [ ]:
# close all sessions / connections to db

sqlalchemy.orm.session.close_all_sessions()

In [ ]:
# inspect a given table

engine.execute( 'SELECT * FROM taxonomy').fetchall()


In [ ]:
# show all tables that exist in db

engine.execute( "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE'").fetchall()


[('threats',),
 ('habitats',),
 ('research_needed',),
 ('use_trade',),
 ('cites_index',),
 ('assessments',),
 ('common_names',),
 ('taxonomy',),
 ('conservation_needed',),
 ('countries',)]

In [ ]:
# show all dbs that exist in...space?

engine.execute( 'SELECT datname FROM pg_database WHERE datistemplate = false').fetchall()


[('rdsadmin',), ('postgres',), ('project_canopy_test',)]

In [ ]:
# csv imports, with compatible (post-cleaning) NaN handling

assessments = pd.read_csv( 'datasets/assessments.csv', low_memory= False, na_filter= False)
commonNames = pd.read_csv( 'datasets/common_names.csv', low_memory= False, na_filter= False)
taxonomy = pd.read_csv( 'datasets/taxonomy.csv', low_memory= False, na_filter= False)
consNeeded = pd.read_csv( 'datasets/conservation_needed.csv', low_memory= False, na_filter= False)
countries = pd.read_csv( 'datasets/countries.csv', low_memory= False, na_filter= False)
habitats = pd.read_csv( 'datasets/habitats.csv', low_memory= False, na_filter= False)
research = pd.read_csv( 'datasets/research_needed.csv', low_memory= False, na_filter= False)
threats = pd.read_csv( 'datasets/threats.csv', low_memory= False, na_filter= False)
useTrade = pd.read_csv( 'datasets/usetrade.csv', low_memory= False, na_filter= False)
citesIndex = pd.read_csv( 'datasets/cites_listings.csv', low_memory= False, na_filter= False)


In [ ]:
habitats.code.value_counts()


1.6     3117
5.1     2611
5.7     1510
5.8     1397
5.5     1291
5.2     1243
1.9     1242
5.4     1238
1.5      764
5.6      626
14.6     396
1.8      365
1.7      192
5.3      169
5.9      156
Name: code, dtype: int64

In [ ]:
# list for local-ish batch uploading to db (below)
# 'file' seems to be an existing (protected?) object name, so.. fiiles ¯\_(ツ)_/¯

fiiles = ['datasets/assessments.csv',
          'datasets/taxonomy.csv',
          'datasets/common_names.csv',
          'datasets/conservation_needed.csv',
          'datasets/countries.csv',
          'datasets/habitats.csv',
          'datasets/research_needed.csv',
          'datasets/threats.csv',
          'datasets/usetrade.csv',
          'datasets/cites_listings.csv',
]

In [ ]:
# raw connection allows easier access to DBAPI, use of copy_expert and SQL commands

engineRaw = DB.create_engine( URL, {}).raw_connection()     # via psycopg, instead of sqla
curseRaw = engineRaw.cursor()

#curseRaw.connection.close()

In [ ]:
# batch upload from list

for fiile in fiiles:
    base = os.path.basename( fiile)
    tableName = os.path.splitext( base)[0]
    with open( str( fiile), 'r') as f:
        cmd = 'COPY ' + tableName + ' FROM STDIN WITH (FORMAT CSV, HEADER);'
        curseRaw.copy_expert( cmd, f)
    engineRaw.commit()
    print( '{} complete!'.format( tableName))


In [ ]:
# file-by-file upload (can functionize?)

sourceFile = 'datasets/taxonomy.csv'
tableDest = 'taxonomy'

with open( sourceFile, 'r') as f:
    cmd = 'COPY ' + tableDest + ' FROM STDIN WITH (FORMAT CSV, HEADER);'
    curseRaw.copy_expert( cmd, f)
    engineRaw.commit()

In [ ]:
## PROBLEM TABLES ##
## list of tables that threw errors on upload attempt, source of error

'''
common_names --\
countries       \ 
research_needed   -->   change pri key to additional unique index value, fill index column
habitats        /
threats        /        ## done ##
use_trade   --/
'''


In [ ]:
# fix for above: write existing (clean) tables to new CSVs -- by default, to_csv
# wants to create / pre-pend an (unnamed) index column, so just forced the name

commonNames.to_csv( 'common_names_new.csv', index_label= 'index')
consNeeded.to_csv( 'conservation_needed_new.csv', index_label= 'index')
countries.to_csv( 'countries_new.csv', index_label= 'index')
habitats.to_csv( 'habitats_new.csv', index_label= 'index')
research.to_csv( 'research_needed_new.csv', index_label= 'index')
threats.to_csv( 'threats_new.csv', index_label= 'index')
useTrade.to_csv( 'usetrade_new.csv', index_label= 'index')
citesIndex.to_csv( 'citesIndex_new.csv', index_label= 'index')


In [ ]:
# fix for above (part 2): after new CSVs are created, need to re-instantiate them
# with CORRECT NaN HANDLING THIS TIME
# na_filter= False is a little heavy-handed, but so much time was spent cleaning these tables,
# the probability of it causing problems is low ( !!_FOR NOW_!! )

commonNames_new = pd.read_csv( 'common_names_new.csv', low_memory= False, na_filter= False)
consNeeded_new = pd.read_csv( 'conservation_needed_new.csv', low_memory= False, na_filter= False)
countries_new = pd.read_csv( 'countries_new.csv', low_memory= False, na_filter= False)
habitats_new = pd.read_csv( 'habitats_new.csv', low_memory= False, na_filter= False)
research_new = pd.read_csv( 'research_needed_new.csv', low_memory= False, na_filter= False)
threats_new = pd.read_csv( 'threats_new.csv', low_memory= False, na_filter= False)
useTrade_new = pd.read_csv( 'usetrade_new.csv', low_memory= False, na_filter= False)
citesIndex_new = pd.read_csv( 'citesIndex_new.csv', low_memory= False, na_filter= False)
